In [1]:

import pandas as pd
import numpy as np
from numpy import random
from sklearn.model_selection import train_test_split
import ast
import os
from PIL import Image
import yaml

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd

# CSV 파일 로드 함수
def load_data_with_pandas(file_path):
    data = pd.read_csv(file_path)
    return data

# 데이터 로드
data_bumper = load_data_with_pandas('./data/data/output/bumper/bumper_data.csv')
data_door_ed = load_data_with_pandas('./data/data/output/door/door_ed_data.csv')
data_door_scratch = load_data_with_pandas('./data/data/output/door/door_scratch_data.csv')
data_fender = load_data_with_pandas('./data/data/output/fender/fender_ed_data.csv')
data_frame_ed = load_data_with_pandas('./data/data/output/frame/frame_ed_data.csv')
data_frame_hd = load_data_with_pandas('./data/data/output/frame/frame_hd_data.csv')
data_frame_sealf = load_data_with_pandas('./data/data/output/frame/frame_sealf_data.csv')
data_frame_seamf = load_data_with_pandas('./data/data/output/frame/frame_seamf_data.csv')

# 데이터 결합
data = pd.concat([
    data_bumper,
    data_door_ed,
    data_door_scratch,
    data_fender,
    data_frame_ed,
    data_frame_hd,
    data_frame_sealf,
    data_frame_seamf
], axis=0)

# y에는 라벨, x에는 나머지 특성 정보
y = data['quality']

# Stratified Split: 학습 데이터와 검증 데이터 비율을 80:20으로 설정, 클래스 비율을 유지
train_data, test_data = train_test_split(data, test_size=0.2, stratify=y, random_state=60)

# 학습 데이터에서 다시 80:20 비율로 나누어 검증 데이터 분리
train_data_80, valid_data = train_test_split(train_data, test_size=0.2, stratify=train_data['quality'], random_state=60)

# CSV로 저장
train_data_80.to_csv('./data/train_data.csv', index=False)
test_data.to_csv('./data/test_data.csv', index=False)
valid_data.to_csv('./data/valid_data.csv', index=False)

# 결과 출력
print(f"총 데이터 개수: {len(data)}")
print(f"학습 데이터 개수: {len(train_data_80)}")
print(f"테스트 데이터 개수: {len(test_data)}")
print(f"검증 데이터 개수: {len(valid_data)}")


In [3]:
# # 빈 리스트의 개수를 세는 코드
# empty_bbox_count = sum(1 for bbox in train_data["bboxes"] if bbox == '[]')

# # 결과 출력
# print(f"빈 리스트의 개수: {empty_bbox_count}")


# # 리스트의 개수를 세는 코드
# bbox_count = sum(1 for bbox in train_data["bboxes"] if not bbox == '[]')

# # 결과 출력
# print(f"리스트의 개수: {bbox_count}")



# 테스트 데이터

In [4]:
# Step 1: 학습 데이터에서 20% 추출
train_20, _ = train_test_split(train_data_80, test_size=0.8, random_state=35)

# Step 2: 검증 데이터에서 20% 추출
valid_20, _ = train_test_split(valid_data, test_size=0.8, random_state=35)

# Step 3: 테스트 데이터에서 20% 추출
test_20, _ = train_test_split(test_data, test_size=0.8, random_state=35)

# Step 4: 각각의 20%에서 다시 20% 추출 (성능 검증용 데이터 생성)
train_test_20, _ = train_test_split(train_20, test_size=0.8, random_state=42)  # 학습 데이터 20%에서 20% 추출
test_test_20, _ = train_test_split(test_20, test_size=0.8, random_state=42)  # 테스트 데이터 20%에서 20% 추출
valid_test_20, _ = train_test_split(valid_20, test_size=0.8, random_state=42)  # 검증 데이터 20%에서 20% 추출

# CSV로 저장
train_test_20.to_csv('./data/test/train_test_20.csv', index=False)
test_test_20.to_csv('./data/test/test_test_20.csv', index=False)
valid_test_20.to_csv('./data/test/valid_test_20.csv', index=False)


In [ ]:
print(len(train_test_20))
print(len(test_test_20))
print(len(valid_test_20))

In [6]:
# 이미지 복사
import shutil

def copy_images(data, output_dir):
    image_dir = os.path.join(output_dir, 'images')
    os.makedirs(image_dir, exist_ok=True)

    for index, row in data.iterrows():
        image_path = row['path']
        shutil.copy(image_path, image_dir)


# 학습 데이터와 검증 데이터를 각각 처리
copy_images(train_test_20, './train')
copy_images(test_test_20, './test')
copy_images(valid_test_20,'./valid')

### bboxes 데이터 추출

In [ ]:
import ast

def parse_bboxes(data):
    """
    문자열 형태의 bounding box 정보를 파싱하여 리스트 형태로 변환하는 함수

    Args:
        data: 파싱할 데이터셋

    Returns:
        list: 파싱된 bounding box 리스트
    """

    bboxes_list = []
    for bbox_str in data["bboxes"]:
        try:
            bboxes = ast.literal_eval(bbox_str)
            bbox_list = []
            for bbox in bboxes:
                x, y, w, h = bbox
                bbox_list.append([x, y, w, h])
            bboxes_list.append(bbox_list)
        except (ValueError, SyntaxError) as e:
            print(f"Error parsing bbox: {e}")
    return bboxes_list

# 데이터셋별 bounding box 파싱
train_bboxes = parse_bboxes(train_test_20)
valid_bboxes = parse_bboxes(valid_test_20)
test_bboxes = parse_bboxes(test_test_20)

# 결과 출력 (예시)
print(train_bboxes[:5])
print(valid_bboxes[:5])
print(test_bboxes[:5])

# .yaml label 데이터 생성

### 1. 디렉토리 경로 전처리

In [ ]:
import os
from PIL import Image


def extract_image_info(image_paths):
    """
    이미지 경로 리스트에서 이미지 크기, 디렉토리 경로, 파일 이름을 추출하는 함수

    Args:
        image_paths (list): 이미지 경로 리스트

    Returns:
        tuple: (image_sizes, image_dirs, file_names)
            image_sizes (list): 이미지 크기 리스트
            image_dirs (list): 이미지 디렉토리 경로 리스트
            file_names (list): 이미지 파일 이름 리스트
    """

    image_sizes = []
    image_dirs = []
    file_names = []

    for path in image_paths:
        try:
            image = Image.open(path)
            width, height = image.size
            image_sizes.append((width, height))
            image_dirs.append(os.path.dirname(path) + os.sep)
            file_names.append(os.path.splitext(os.path.basename(path))[0])
        except FileNotFoundError:
            print(f"파일을 찾을 수 없습니다: {path}")

    return image_sizes, image_dirs, file_names


# 데이터 준비 (예시)
train_image_paths = train_test_20.iloc[:, 0].tolist()
valid_image_paths = valid_test_20.iloc[:, 0].tolist()
test_image_paths = test_test_20.iloc[:, 0].tolist()

# 이미지 정보 추출
train_sizes, train_dirs, train_names = extract_image_info(train_image_paths)
valid_sizes, valid_dirs, valid_names = extract_image_info(valid_image_paths)
test_sizes, test_dirs, test_names = extract_image_info(test_image_paths)

# 결과 출력
print("Train 이미지 크기:", train_sizes[:5])
print("Train 이미지 디렉토리:", train_dirs[:5])
print("Train 파일 이름:", train_names[:5])
print()
print("Validation 이미지 크기:", valid_sizes[:5])
print("Validation 이미지 디렉토리:", valid_dirs[:5])
print("Validation 파일 이름:", valid_names[:5])
print()
print("test 이미지 크기:", test_sizes[:5])
print("test 이미지 디렉토리:", test_dirs[:5])
print("test 파일 이름:", test_names[:5])

### 3. 부품 상태 전처리

In [ ]:
# 부품 상태를 체크하는 함수
def get_label(part, status):
    if part == "범퍼":
        if status == 1:
            return 0
        elif status == 0:
            return 1
    elif part == "도어":
        if status == 1:
            return 0
        elif status == 0:
            return 2
    elif part == "휀더":
        if status == 1:
            return 0
        elif status == 0:
            return 3
    elif part == "프레임":
        if status == 1:
            return 0
        elif status == 0:
            return 4
    return None  # 기본적으로 반환할 값이 없으면 None


# 예시 부품과 상태
parts_and_status_train = []
parts_and_status_valid = []
parts_and_status_test = []
for quality, part in zip(train_test_20.iloc[:, 1], train_test_20.iloc[:, 2]):
    parts_and_status_train.append({"part": part, "status": quality})

for quality, part in zip(test_test_20.iloc[:, 1], test_test_20.iloc[:, 2]):
    parts_and_status_test.append({"part": part, "status": quality})

for quality, part in zip(valid_test_20.iloc[:, 1], valid_test_20.iloc[:, 2]):
    parts_and_status_valid.append({"part": part, "status": quality})

print(f"parts_and_status_train : {parts_and_status_train}")
print(f"parts_and_status_test : {parts_and_status_test}")
print(f"parts_and_status_valid : {parts_and_status_valid}")

# 라벨링 결과를 저장할 리스트
labels_train = []
labels_valid = []
label_test = []

# 각 부품 상태에 대해 라벨을 설정
for item in parts_and_status_train:
    label = get_label(item["part"], item["status"])
    labels_train.append(label)
for item in parts_and_status_valid:
    label = get_label(item["part"], item["status"])
    labels_valid.append(label)
for item in parts_and_status_test:
    label = get_label(item["part"], item["status"])
    label_test.append(label)

# # 결과 출력
print("train 라벨링 결과 :", labels_train)
print("valid 라벨링 결과 :", labels_valid)
print("test 라벨링 결과 :", label_test)

In [10]:
def convert_to_yolo_format(bbox, label, image_width, image_height):
    yolo_labels = []  # 각 이미지에 대한 라벨을 담을 리스트

    if not bbox:
        # bbox가 비어있으면 기본값으로 라벨을 설정
        yolo_labels.append(f"{label} 0.5 0.5 0.0 0.0")
    else:
        # bbox가 있으면 여러 개의 파손 부위를 처리
        for x_min, y_min, width, height in bbox:
            # YOLO 형식에 맞게 좌표를 정규화
            x_center = (x_min + width / 2) / image_width
            y_center = (y_min + height / 2) / image_height
            norm_width = width / image_width
            norm_height = height / image_height
            # 정규화된 좌표와 함께 라벨을 저장
            yolo_labels.append(f"{label} {x_center:.6f} {y_center:.6f} {
                               norm_width:.6f} {norm_height:.6f}")

    return yolo_labels


def save_yolo_labels(output_dir, bboxes, labels, size, names):
    # 각 이미지에 대해 라벨을 텍스트 파일로 저장
    for bbox, label, (w, h), name in zip(bboxes, labels, size, names):
        yolo_labels = convert_to_yolo_format(bbox, label, w, h)

        # 라벨을 .txt 파일로 저장
        txt_file_path = os.path.join(output_dir, f"{name}.txt")
        with open(txt_file_path, 'w') as txt_file:
            for yolo_data in yolo_labels:
                txt_file.write(f"{yolo_data}\n")



# 라벨을 저장할 디렉토리
train_output_dir = './train/labels'
os.makedirs(train_output_dir, exist_ok=True)

# YOLO 라벨 저장
save_yolo_labels(train_output_dir, train_bboxes, labels_train, train_sizes, train_names)

# 라벨을 저장할 디렉토리
valid_output_dir = './valid/labels'
os.makedirs(valid_output_dir, exist_ok=True)

# YOLO 라벨 저장
save_yolo_labels(valid_output_dir, valid_bboxes, labels_valid, valid_sizes, valid_names)


# 라벨을 저장할 디렉토리
test_output_dir = './test/labels'
os.makedirs(test_output_dir, exist_ok=True)

# YOLO 라벨 저장
save_yolo_labels(test_output_dir, test_bboxes, label_test, test_sizes, test_names)

In [11]:
# YAML 파일에 저장
yaml_data = {
    "train": "C:/Users/dawoo/desktop/sf7/coding/pytorch/project/train/images",
    "test": "C:/Users/dawoo/desktop/sf7/coding/pytorch/project/test/images",
    "val": "C:/Users/dawoo/desktop/sf7/coding/pytorch/project/valid/images",
    "nc": 5,
    "names": ["ok",
              "bumper_faulty",
              "door_faulty",
              "fender_faulty",
              "frame_faulty"]
}

with open(f"data.yaml", "w") as yaml_file:
    yaml.dump(yaml_data, yaml_file, default_flow_style=False, sort_keys=False)

# print("YAML 파일이 생성되었습니다.")

## 모델 코드 작성

In [12]:
# from ultralytics import YOLO

# # YOLO 모델 객체 생성
# model = YOLO("yolov8s.pt")  # 사전 학습된 YOLOv8 모델 로드

# # 모델 학습 시작
# results = model.train(
#     data="data.yaml",       # YAML 파일 경로
#     epochs=20,              # 학습 에포크 수
#     imgsz=640,              # 입력 이미지 크기 (기본값: 640)
#     batch=16,               # 배치 크기
#     project="C:\\Users\\dawoo\\Desktop\\SF7\\coding\\PYTORCH\\project\\YOLO", # 학습 결과를 저장할 프로젝트 이름
#     name="part_damage_detection",   # 저장될 실행 이름
#     workers=4,              # 데이터 로드 병렬 처리 수
#     device=0                # GPU ID (0은 첫 번째 GPU를 의미, -1은 CPU)
# )


In [13]:
# import os
# from ultralytics import YOLO

# # 데이터 설정 파일 경로
# data_yaml = "data.yaml"

# # YOLO 모델 정의 (랜덤 초기화로 학습 시작)
# model = YOLO("yolov8s.pt")
# # model.model = model.model.reset_parameters()  # 가중치 초기화 (랜덤 가중치)

# # 학습 설정
# model.train(
#     data=data_yaml,          # 데이터 설정 파일 경로
#     epochs=50,               # 학습 에폭 수
#     batch=16,                # 배치 크기
#     imgsz=640,               # 입력 이미지 크기
#     device=0,                # GPU 사용
#     workers=4,               # 데이터 로드 병렬 처리 워커 수
#     project="C:\\Users\\dawoo\\Desktop\\SF7\\coding\\PYTORCH\\project\\YOLO",     # 결과 저장 디렉토리
#     name="part_damage_detection",       # 실험 이름
#     conf=0.5,
#     lr0=0.001,  # 초기 학습률
#     optimizer='AdamW',  # 옵티마이저 설정 (SGD, Adam, AdamW 등)
#     weight_decay=0.01   # 가중치 감소 설정
# )
# model.save('part_damage_detection.pt')

In [14]:
# import os
# from ultralytics import YOLO

# # 데이터 설정 파일 경로
# data_yaml = "data.yaml"

# # YOLO 모델 정의 (yolov8m.pt 사용)
# model = YOLO("yolov8n.pt")

# # 학습 설정
# model.train(
#     data=data_yaml,
#     epochs=70,
#     batch=8,  # 배치 크기 감소
#     imgsz=640,  # 이미지 크기
#     device=0,
#     workers=3,
#     project="YOLO1",
#     name="part_damage_detection",
#     conf=0.5,
#     lr0=0.0001,
#     optimizer='AdamW',
#     weight_decay=0.01,
#     patience=10,
# )

# model.save('part_damage_detection2.pt')

In [ ]:
import os
from ultralytics import YOLO

# 데이터 설정 파일 경로
data_yaml = "data.yaml"

# YOLO 모델 정의 (yolov8m.pt 사용)
model = YOLO("yolov8s.pt")

# 학습 설정
model.train(
    data=data_yaml,
    epochs=50,
    batch=8,  # 배치 크기 감소
    imgsz=640,  # 이미지 크기 줄이기
    device=0,
    workers=4,
    project="YOLO2",
    name="part_damage_detection",
    conf=0.5,
    lr0=0.0005,
    optimizer='AdamW',
    weight_decay=0.005,
    patience=10,
)

model.save('part_damage_detection3.pt')

In [ ]:
# 모델 평가
metrics = model.val()

In [ ]:
# 모델 로드 및 추론
model = YOLO("C:\\Users\\dawoo\\desktop\\sf7\\coding\\pytorch\\project\\model_8_s_50_16_944.pt")
results = model.predict(source="C:\\Users\\dawoo\\desktop\\sf7\\coding\\pytorch\\project\\test\\images", save=True, imgsz=640)



In [ ]:
count = 0
for result in results:
    # 바운딩 박스가 없는 경우 'no detections'으로 간주
    if result.boxes is None or len(result.boxes.xywh) == 0:
        count += 1

print(f"no detections : {count}")
print(f"len(results) : {len(results)}")


In [3]:
import cv2
from ultralytics import YOLO

# YOLOv8 모델 로드
model_path = "C:\\Users\\dawoo\\desktop\\sf7\\coding\\pytorch\\project\\model_8_s_50_16_944.pt"  # 학습한 모델 경로
model = YOLO(model_path)

# 카메라 열기
cap = cv2.VideoCapture(0)  # 기본 카메라 (ID 0)
if not cap.isOpened():
    print("카메라를 열 수 없습니다.")
    exit()

# 카메라 프레임 크기 설정 (선택 사항)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# 색상 설정 (바운딩 박스와 텍스트)
box_color = (0, 255, 0)  # 녹색
text_color = (255, 255, 255)  # 흰색

print("Press 'q' to exit...")

while True:
    # 프레임 읽기
    ret, frame = cap.read()
    if not ret:
        print("프레임을 읽을 수 없습니다.")
        break
    flip_frame = cv2.flip(frame, 1)
    # 모델 추론 (YOLOv8)
    results = model(source=frame, conf=0.5, show=False,
                    verbose=False)  # 이미지에 대해 추론
    detections = results[0]  # 첫 번째 결과 가져오기

    # 바운딩 박스와 라벨 그리기
    for box in detections.boxes:
        # 바운딩 박스 좌표 (정규화된 값 아님)
        x1, y1, x2, y2 = map(int, box.xyxy[0])  # 좌상단(x1, y1), 우하단(x2, y2)
        conf = box.conf[0]  # 신뢰도
        cls = int(box.cls[0])  # 클래스 ID
        label = f"{model.names[cls]} {conf:.2f}"  # 클래스 이름과 신뢰도

        # 바운딩 박스와 라벨 추가
        cv2.rectangle(frame, (x1, y1), (x2, y2), box_color, 2)  # 바운딩 박스
        cv2.putText(frame, label, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 2)

    # 프레임 화면에 표시
    cv2.imshow("YOLOv8 Real-Time Detection", flip_frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 자원 해제
cap.release()
cv2.destroyAllWindows()

Press 'q' to exit...


## 손실 그래프

In [ ]:
# loss function
import matplotlib.pyplot as plt

# 손실 감소 그래프
plt.subplot(1, 2, 1)
plt.plot(losses)
plt.grid()
plt.title('Loss over Epochs')
plt.xlabel('epoch')
plt.ylabel('Loss')

## 데이터 검증

In [ ]:
## data evaluation

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_curve, roc_auc_score
pred=model(y_test)


# 혼동 행렬
confmat = confusion_matrix(y_true=y_test, y_pred=pred)
print('혼동 행렬: ', confmat)


# 정확도
accuracy=accuracy_score(y_true=y_test, y_pred=pred)
print('정확도: ', accuracy)


# 정밀도(precision)
precision = precision_score(y_true=y_test, y_pred=pred)
print('정밀도: ', precision)


# 재현율(recall)
recall = recall_score(y_true=y_test, y_pred=pred)
print('재현율: ', recall)


# roc auc
fpr, tpr, _ = roc_curve(y_true=y_test, y_pred=pred)

plt.plot(fpr, tpr)
plt.xlabel('FP Rate')
plt.ylabel('TP Rate')

plt.show()

print('auc 점수: ', roc_auc_score(y_test, pred))